## Have a look into the sample file

This file is just an example. You could find something similar e.g. in the header of a CSV file.

In this notebook we will only handle metadata.

In [ ]:
SAMPLE_FILE = 'sample.txt'

In [ ]:
with open(SAMPLE_FILE, 'r') as metafile:
    for l in metafile:
        print(l)

## Parse the sample file

In [ ]:
with open(SAMPLE_FILE, 'r') as metafile:
    meta = {k: v.strip() for k, v in (l.split('=') for l in metafile)}

In [ ]:
meta

## Setup eLabFTW API

In [ ]:
import elabapi_python

`API_KEY` and `API_HOST_URL` need to be set for your individual environment.

Your API Host URL could look something like `https://my-elab.university.com/api/v2`.

Your API Key is a personalised secret. Whoever owns it, has access as if they had your username and password. **Keep it safe!** To generate an API Key:

- log in to your eLabFTW instance,
- go to the user panel via the button in the top right,
- switch to the API Keys tab,
- and create a new key.

In [ ]:
API_KEY = '123abc456def'
API_HOST_URL = 'https://my-elab.university.com/api/v2'

In [ ]:
# This is not required for the example, but made my life easier during development, to make sure I don't publish my URLs and keys by accident.
from dotenv import dotenv_values
config = dotenv_values(".env")
API_KEY = config.get('API_KEY', API_KEY)
API_HOST_URL = config.get('API_HOST_URL', API_HOST_URL)

In [ ]:
configuration = elabapi_python.Configuration()
configuration.api_key['api_key'] = API_KEY
configuration.api_key_prefix['api_key'] = 'Authorization'
configuration.host = API_HOST_URL
configuration.debug = False
configuration.verify_ssl = True

In [ ]:
api_client = elabapi_python.ApiClient(configuration)
api_client.set_default_header(header_name='Authorization', header_value=API_KEY) # fix issue with Authorization header not being properly set by the generated lib
experiments_api = elabapi_python.ExperimentsApi(api_client)

## Create a new, empty experiment

In [ ]:
response = experiments_api.post_experiment_with_http_info()

Have a look into your experiments. You should now find an untitled experiment.

### Have a look at the result in Python

You got back an answer from eLabFTW. If you look inside, you can find e.g. the API URL that directly accesses the new experiment. In it, you can see the new experiment's id at the end. We will filter out that id to you use it in the following steps.

In [ ]:
import pprint

In [ ]:
pprint.pprint({k:v for k,v in zip(response[2].keys(), response[2].values())})

In [ ]:
def filter_experiment_number(response):
    """
    Extract the experiment id from the response.

    :param response: The response from the API containing the experiment link.
    :return: The extracted experiment number.
    """
    location = response[2]['Location']
    for i in range(len(location) - 1, -1, -1):
        if location[i] == '/':
            experiment_number = location[i+1:]
            break
    return experiment_number

In [ ]:
experiment_id = filter_experiment_number(response)

In [ ]:
experiment_id

## Give the new experiment a title

In [ ]:
response = experiments_api.patch_experiment(experiment_id, body={"title": 'A nice new experiment'})

Have a look into your experiment in eLabFTW.

If you want, you can also have a look into the response:

In [ ]:
response

## Add the metadata from the sample file to your new experiment

We will use the Python built-in JSON package to generate a JSON string from the metadata object `meta` (actually it is a dict), that we created earlier.

In [ ]:
import json

Now, add that to your experiment via the eLabFTW API.

In [ ]:
response = experiments_api.patch_experiment(experiment_id, body={"metadata": json.dumps(meta)})

Have a look into your experiment in eLabFTW.

If you want, you can also have a look into the response:

In [ ]:
response

# That's it!